In [6]:
import pickle
import pandas as pd
import os
import sys
sys.path.insert(0, "/home/ubuntu/data/code/Modules/")
import glue_code
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier,BaggingClassifier,GradientBoostingClassifier

In [9]:
test_preds = pickle.load(open('/home/ubuntu/data/code/2dCNN+Fluctuations/Analysis_Results/11_12_test_predictions.p','rb'))
train_preds = pickle.load(open('/home/ubuntu/data/code/2dCNN+Fluctuations/Analysis_Results/11_12_train_predictions.p','rb'))

In [18]:
#we have predictions on every image, need to find test and train cells
test_cells = sorted(glue_code.partition_to_keys(list(test_preds.keys())))
train_cells = sorted(glue_code.partition_to_keys(list(train_preds.keys())))

In [19]:
#go from list of cells, to list of lists of images from each cell
cell_ims_test = glue_code.split_preds_into_cells(test_preds,test_cells)
cell_ims_train = glue_code.split_preds_into_cells(train_preds,train_cells)

#get the av score for susceptible, and proportion of images classified as susceptible
cell_predictions_test = glue_code.get_cell_predictions(cell_ims_test,test_preds)
cell_predictions_train = glue_code.get_cell_predictions(cell_ims_train,train_preds)

In [23]:
cell_predictions_train.extend(cell_predictions_test)
all_preds = np.copy(cell_predictions_train)

In [36]:
all_preds

array([['/home/ubuntu/data/control/sample02/cell272', '0.0',
        '0.0069812946'],
       ['/home/ubuntu/data/control/sample02/cell273', '1.0', '0.9810025'],
       ['/home/ubuntu/data/control/sample02/cell274',
        '0.6666666666666666', '0.5327192'],
       ...,
       ['/home/ubuntu/data/susceptible/sample07/cell302', '1.0',
        '0.99920577'],
       ['/home/ubuntu/data/susceptible/sample07/cell312', '1.0',
        '0.99859697'],
       ['/home/ubuntu/data/susceptible/sample07/cell323', '1.0',
        '0.9999998']], dtype='<U46')

In [47]:
preds = np.array(all_preds)[:,1:].astype('float')
pred_dict = {}
for i,name in enumerate(all_preds):
    pred_dict[name[0]]=preds[i]

In [107]:
samples = []
sample_pred_dict={}
for key in pred_dict.keys():#move through keys in pred dict
    sample_path = key[:-7] #find the sample the cell belongs to
    if sample_path in sample_pred_dict.keys(): #if we have this sample in the sample list
        sample_pred_dict[sample_path]=np.vstack((sample_pred_dict[sample_path],pred_dict[key])) #stack the new prediction
    else:
        sample_pred_dict[sample_path]=pred_dict[key]

In [117]:
final_dict={}
for key in sample_pred_dict.keys():
    scores = np.average(sample_pred_dict[key],axis=0)
    final_dict[key]=scores

In [118]:
final_dict

{'/home/ubuntu/data/control/sample02/': array([0.15925926, 0.19408252]),
 '/home/ubuntu/data/control/sample03/': array([0.0620915 , 0.11432719]),
 '/home/ubuntu/data/control/sample04/': array([0.05381944, 0.09293261]),
 '/home/ubuntu/data/control/sample05/': array([0.03146259, 0.10381063]),
 '/home/ubuntu/data/control/sample06/': array([0.14460784, 0.19266409]),
 '/home/ubuntu/data/control/sample07/': array([0.0245098 , 0.02487492]),
 '/home/ubuntu/data/control/sample08/': array([0.00490196, 0.02388881]),
 '/home/ubuntu/data/control/sample09/': array([0.06862745, 0.20401009]),
 '/home/ubuntu/data/control/sample10/': array([0.03104575, 0.10800065]),
 '/home/ubuntu/data/control/sample11/': array([0.02636054, 0.07122009]),
 '/home/ubuntu/data/control/sample12/': array([0.16753472, 0.24867582]),
 '/home/ubuntu/data/control/sample13/': array([0.1925    , 0.27943614]),
 '/home/ubuntu/data/susceptible/sample01/': array([0.68560606, 0.66079201]),
 '/home/ubuntu/data/susceptible/sample02/': arr

In [116]:
sample_pred_dict

{'/home/ubuntu/data/control/sample02/': array([[0.00000000e+00, 6.98129460e-03],
        [1.00000000e+00, 9.81002500e-01],
        [6.66666667e-01, 5.32719200e-01],
        [0.00000000e+00, 6.51170800e-04],
        [0.00000000e+00, 6.58814900e-02],
        [0.00000000e+00, 9.09878800e-06],
        [0.00000000e+00, 1.03242510e-04],
        [0.00000000e+00, 2.09806280e-01],
        [0.00000000e+00, 1.12106510e-01],
        [0.00000000e+00, 1.34767740e-03],
        [3.33333333e-01, 4.55164880e-01],
        [0.00000000e+00, 9.25636000e-09],
        [0.00000000e+00, 2.29791180e-03],
        [0.00000000e+00, 1.20667280e-01],
        [0.00000000e+00, 7.22000900e-02],
        [0.00000000e+00, 1.86350350e-02],
        [0.00000000e+00, 1.28768310e-01],
        [0.00000000e+00, 4.38808600e-04],
        [0.00000000e+00, 5.06286000e-05],
        [0.00000000e+00, 4.72247780e-05],
        [0.00000000e+00, 5.61081600e-03],
        [0.00000000e+00, 1.50112260e-01],
        [0.00000000e+00, 5.10614400e-

In [95]:
pred_dict[key][...,None].shape

(2, 1)

In [96]:
sample_pred_dict[sample_path].shape

(2, 2)